In [7]:
import os, sys
import keras
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = 'C:\\Users\\carellano.LEGION5PRO-O3OB\\Document\\GitHub\\Mask_RCNN'
# assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

In [2]:

model_path = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet_v1\\mask_rcnn_spacenet_0151\\mask_rcnn_spacenet_0151.h5"
new_model_path = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet_v1\\transferlearning\\"


In [8]:
class SpaceNetConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SpaceNet"
    BACKBONE = "resnet50"
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 building

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_RATIOS = [0.25, 1, 4]

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 5  #10    #32

    USE_MINI_MASK = True
    
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 50 #500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 10 #50
    
    MAX_GT_INSTANCES=250
    DETECTION_MAX_INSTANCES=350
    
    
    
    
config = SpaceNetConfig()
config.display()




Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        350
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [9]:
#method1

# Root directory of the project
ROOT_DIR = os.path.abspath("C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet_v1\\")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)


# model = tf.keras.models.load_model(model_path)
# tf.saved_model.save(model, new_model_path)

In [11]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

... done downloading pretrained model!


In [13]:
# load weights (mscoco) and exclude the output layers

model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# model.save("new_model_path")
tf.saved_model.save(model, new_model_path)

ValueError: Expected an object of type `Trackable`, such as `tf.Module` or a subclass of the `Trackable` class, for export. Got <mrcnn.model.MaskRCNN object at 0x00000222ADBA5280> with type <class 'mrcnn.model.MaskRCNN'>.

In [ ]:
#method 2
model = tf.keras.models.load_model(model_path)
# model.save("new_model_path")

In [ ]:
import h5py
h5py.run_tests()

In [ ]:
#method3

method = input("Input the type of model loading (JSON or H5):")
if method == 'JSON':    
    #load acritecture from json file
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json, custom_objects={'BatchNorm':BatchNorm, 'tf':tf, 'ProposalLayer':ProposalLayer, 'PyramidROIAlign1':PyramidROIAlign1, 'PyramidROIAlign2':PyramidROIAlign2, 'DetectionLayer':DetectionLayer})

    # load weights into new model
    model.load_weights("trained_weights.h5", by_name=True)
    print("Loaded model from disk")

elif method == 'H5':
    #load entire model
    model = keras.models.load_model('model.h5', custom_objects={'BatchNorm':BatchNorm, 'tf':tf, 'ProposalLayer':ProposalLayer, 'PyramidROIAlign1':PyramidROIAlign1, 'PyramidROIAlign2':PyramidROIAlign2, 'DetectionLayer':DetectionLayer}, compile=False)
    print("Loaded model from disk")